In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('beauty_data_v5.csv')

In [ ]:
import pandas as pd
import nltk
import re, string, unicodedata, contractions
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from contractions import contractions_dict
from nltk.tokenize import word_tokenize, sent_tokenize, regexp_tokenize 
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# special_characters removal
def remove_special_characters(text, remove_digits=True):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation_and_splchars(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_word = remove_special_characters(new_word, True)
            new_words.append(new_word)
    return new_words

stopword_list= stopwords.words('english')

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopword_list:
            new_words.append(word)
    return new_words

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = to_lowercase(words)
    words = remove_punctuation_and_splchars(words)
    words = remove_stopwords(words)
    return words

def lemmatize(words):
    lemmas = lemmatize_verbs(words)
    return lemmas

In [ ]:
def normalize_and_lemmaize(input):
    sample = remove_special_characters(input)
    words = nltk.word_tokenize(sample)
    words = normalize(words)
    lemmas = lemmatize(words)
    return ' '.join(lemmas)

In [ ]:
df1['clean_text'] = df1['review'].map(lambda text: normalize_and_lemmaize(text))

C:\Users\LENOVO\AppData\Local\Temp/ipykernel_18824/803180331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['clean_text'] = df1['review'].map(lambda text: normalize_and_lemmaize(text))


In [ ]:
group_user = {
    "revID" : [],
    "review" : [],
}

In [ ]:
for i in df1['revID-baru']:
    group_user['revID'].append(i)
    group_user['review'].append(' '.join(df1[df1['revID-baru']==i]['clean_text']))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

n_gram_range = (0, 1)
stop_words = "english"

In [ ]:
group_user['candidates'] = []

In [ ]:
for i in group_user['review']:
    try:
        count = CountVectorizer(ngram_range=n_gram_range, stop_words="english").fit([i])
        candidates = count.get_feature_names()
        group_user['candidates'].append(candidates)
    except ValueError:
        group_user['candidates'].append('nothing')

In [ ]:
group_user["doc_embedding"] = []
group_user["candidate_embeddings"] = []

In [ ]:
from sentence_transformers import SentenceTransformer

# model = SentenceTransformer('all-mpnet-base-v2')
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
for i in group_user["review"] : 
  group_user["doc_embedding"].append(model.encode([i])) 

In [ ]:
for i in group_user["candidates"] : 
  group_user["candidate_embeddings"].append(model.encode(i)) 

In [ ]:
group_user["top5"] = []

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

top_n = 5

model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
for i in range (len(group_user['candidate_embeddings'])):
  distances = cosine_similarity(group_user['doc_embedding'][i], group_user['candidate_embeddings'][i])
  group_user["top5"].append([group_user["candidates"][i][index] for index in distances.argsort()[0][-top_n:]])

In [ ]:
df_keluarin = pd.DataFrame(group_user)

In [ ]:
df_keluarin

,revID,review,candidates,doc_embedding,candidate_embeddings,top5,top10,top15,top20,top25
0,712752,love moisturizer would recommend someone dry s...,"[brand, day, dry, fine, line, love, moisturize...","[[-0.057427134, -0.026186448, 0.07010983, 0.06...","[[-0.090751626, 0.028474936, -0.0028173306, -0...","[skin, dry, serum, wrinkle, moisturizer]","[use, brand, day, night, recommend, skin, dry,...","[love, fine, line, use, brand, day, night, rec...","[love, fine, line, use, brand, day, night, rec...","[love, fine, line, use, brand, day, night, rec..."
1,800841,receive product deadlinei test baby kabuki qua...,"[baby, best, cute, deadlinei, fibers, itthe, k...","[[-0.08553487, -0.0062449994, 0.053392757, 0.0...","[[-0.06259442, 0.014936047, -0.04488664, 0.045...","[smell, kabuki, soft, material, fibers]","[best, baby, product, quality, package, smell,...","[cute, deadlinei, love, receive, test, best, b...","[itthe, cute, deadlinei, love, receive, test, ...","[itthe, cute, deadlinei, love, receive, test, ..."
2,307856,love set great buy price dont wear makeup time...,"[buy, dont, feel, great, love, makeup, price, ...","[[-0.14331532, 0.042714417, 0.09062704, 0.0127...","[[-0.06259998, 0.02419781, -0.03591996, -0.008...","[set, love, price, buy, makeup]","[great, feel, dont, time, wear, set, love, pri...","[great, feel, dont, time, wear, set, love, pri...","[great, feel, dont, time, wear, set, love, pri...","[great, feel, dont, time, wear, set, love, pri..."
3,286233,nice moisturizer natural ingredients parabens ...,"[bite, doesnt, dont, feel, ingredients, ive, m...","[[-0.100264825, -0.01663649, 0.04286729, 0.035...","[[-0.0686122, 0.059835378, 0.009301031, 0.0351...","[watereddown, parabens, ingredients, moisturiz...","[use, natural, substantal, bite, pricey, water...","[ive, nice, feel, dont, need, use, natural, su...","[doesnt, ive, nice, feel, dont, need, use, nat...","[doesnt, ive, nice, feel, dont, need, use, nat..."
4,789275,please research mac hello kitty collection det...,"[actually, authentic, box, buy, close, collect...","[[-0.12547551, -0.0177332, 0.04059715, 0.00401...","[[-0.04820198, 0.0116336, 0.048280094, 0.06902...","[sellers, purchase, lipstick, counterfeit, ret...","[buy, authentic, seller, line, mac, sellers, p...","[collection, label, product, price, fake, buy,...","[color, shadow, shade, box, kitty, collection,...","[view, picture, offer, determine, consist, col..."
...,...,...,...,...,...,...,...,...,...,...
198101,395117,look boars hair brush run across brush read re...,"[boars, brush, buy, car, couple, create, decen...","[[0.025496006, -0.05440187, 0.070586205, -0.02...","[[-0.0032545275, -0.0044628647, -0.022955932, ...","[boars, static, scalp, hair, brush]","[car, dry, model, phillips, stiff, boars, stat...","[review, work, try, look, purse, car, dry, mod...","[order, easy, doesnt, wet, run, review, work, ...","[extremely, like, size, buy, long, order, easy..."
198102,33648,thick long hair love brush sister steal order ...,"[amazon, brush, hair, happy, long, love, order...","[[-0.07098996, 0.025867842, 0.027534354, 0.003...","[[-0.02318193, -0.026144285, -0.06706472, 0.03...","[order, sister, amazon, hair, brush]","[love, happy, steal, long, order, sister, amaz...","[love, happy, steal, long, order, sister, amaz...","[love, happy, steal, long, order, sister, amaz...","[love, happy, steal, long, order, sister, amaz..."
198103,1049744,like product smell nice work well go well drop...,"[baggie, break, brush, cheap, check, crack, dr...","[[-0.092968374, -0.00704652, 0.07140467, 0.000...","[[0.0081776045, 0.038500175, -0.040359184, 0.0...","[leak, soft, smell, hair, brush]","[impress, lid, wet, product, baggie, leak, sof...","[purchase, cheap, girls, pretty, stiff, impres...","[drop, try, make, like, particularly, purchase...","[work, check, love, crack, little, drop, try, ..."
198104,38810,hard find brush feel good thick hair second si...,"[brush, difference, feel, good, hair, hard, ma...","[[-0.097549275, -

In [ ]:
df_keluarin.to_csv('./export_3.csv')

In [ ]:
group_user["top10"] = []

In [ ]:

top_n = 10

for i in range (len(group_user['candidate_embeddings'])):
  distances = cosine_similarity(group_user['doc_embedding'][i], group_user['candidate_embeddings'][i])
  group_user["top10"].append([group_user["candidates"][i][index] for index in distances.argsort()[0][-top_n:]])

In [ ]:
group_user["top15"] = []

In [ ]:

top_n = 15

for i in range (len(group_user['candidate_embeddings'])):
  distances = cosine_similarity(group_user['doc_embedding'][i], group_user['candidate_embeddings'][i])
  group_user["top15"].append([group_user["candidates"][i][index] for index in distances.argsort()[0][-top_n:]])

In [ ]:
group_user["top20"] = []

In [ ]:

top_n = 20

for i in range (len(group_user['candidate_embeddings'])):
  distances = cosine_similarity(group_user['doc_embedding'][i], group_user['candidate_embeddings'][i])
  group_user["top20"].append([group_user["candidates"][i][index] for index in distances.argsort()[0][-top_n:]])

In [ ]:
group_user["top25"] = []

In [ ]:

top_n = 25

for i in range (len(group_user['candidate_embeddings'])):
  distances = cosine_similarity(group_user['doc_embedding'][i], group_user['candidate_embeddings'][i])
  group_user["top25"].append([group_user["candidates"][i][index] for index in distances.argsort()[0][-top_n:]])

# Combine

In [ ]:
df2 = pd.read_csv('export_3.csv')

In [ ]:
df2 = df2.drop(['Unnamed: 0'], axis=1)

In [ ]:
df2.isnull().sum()

revID                   0
review                  0
candidates              0
doc_embedding           0
candidate_embeddings    0
top5                    0
top10                   0
top15                   0
top20                   0
top25                   0
dtype: int64

In [ ]:
df.rename(columns = {'revID-baru':'revID'}, inplace = True)

In [ ]:
df['clean_text'] = df['review'].map(lambda text: normalize_and_lemmaize(text))

In [ ]:
df.rename(columns = {'review':'review_lama'}, inplace = True)

In [ ]:
df.rename(columns = {'clean_text':'review'}, inplace = True)

In [ ]:
df.isnull().sum()

reviewerID        0
productID         0
reviewerName      0
review_lama       0
rating            0
reviewTime        0
description       0
title             0
imageUrl          0
salesRank         0
price             0
related           0
brand             0
pos_feedback      0
neg_feedback      0
revID             0
productID-baru    0
review            0
dtype: int64

In [ ]:
df_merge = pd.merge(df,df2,on=['revID', 'review'])

In [ ]:
df_merge.to_csv('export_merge.csv')

In [ ]:
df_merge = pd.read_csv('export_merge.csv')

In [ ]:
df_merge

,Unnamed: 0,reviewerID,productID,reviewerName,review_lama,rating,reviewTime,description,title,imageUrl,...,productID-baru,review,candidates,doc_embedding,candidate_embeddings,top5,top10,top15,top20,top25
0,0,A39HTATAQ9V7YF,0205616461,cheryl roberts,i do love this moisturizer and would recommend...,5,"05 28, 2013","As we age, our once youthful, healthy skin suc...",Bio-Active Anti-Aging Serum (Firming Ultra-Hyd...,http://ecx.images-amazon.com/images/I/41DecrGO...,...,0,love moisturizer would recommend someone dry s...,"['brand', 'day', 'dry', 'fine', 'line', 'love'...",[[-5.74271344e-02 -2.61864476e-02 7.01098293e...,[[-0.09075163 0.02847494 -0.00281733 ... -0.0...,"['skin', 'dry', 'serum', 'wrinkle', 'moisturiz...","['use', 'brand', 'day', 'night', 'recommend', ...","['love', 'fine', 'line', 'use', 'brand', 'day'...","['love', 'fine', 'line', 'use', 'brand', 'day'...","['love', 'fine', 'line', 'use', 'brand', 'day'..."
1,1,A3JM6GV9MNOF9X,0558925278,Patty,I received this product before the deadline.I ...,3,"12 14, 2012",Mineral Powder Brush--Apply powder or mineral ...,Eco Friendly Ecotools Quality Natural Bamboo C...,http://ecx.images-amazon.com/images/I/51L%2BzY...,...,1,receive product deadlinei test baby kabuki qua...,"['baby', 'best', 'cute', 'deadlinei', 'fibers'...",[[-8.55348706e-02 -6.24499936e-03 5.33927567e...,[[-6.2594421e-02 1.4936047e-02 -4.4886641e-02...,"['smell', 'kabuki', 'soft', 'material', 'fibers']","['best', 'baby', 'product', 'quality', 'packag...","['cute', 'deadlinei', 'love', 'receive', 'test...","['itthe', 'cute', 'deadlinei', 'love', 'receiv...","['itthe', 'cute', 'deadlinei', 'love', 'receiv..."
2,2,A1Z513UWSAAO0F,0558925278,Renita M,I love this set. Great buy for the price. I do...,5,"07 7, 2014",Mineral Powder Brush--Apply powder or mineral ...,Eco Friendly Ecotools Quality Natural Bamboo C...,http://ecx.images-amazon.com/images/I/51L%2BzY...,...,1,love set great buy price dont wear makeup time...,"['buy', 'dont', 'feel', 'great', 'love', 'make...",[[-1.43315315e-01 4.27144170e-02 9.06270370e...,[[-0.06259998 0.02419781 -0.03591996 ... -0.0...,"['set', 'love', 'price', 'buy', 'makeup']","['great', 'feel', 'dont', 'time', 'wear', 'set...","['great', 'feel', 'dont', 'time', 'wear', 'set...","['great', 'feel', 'dont', 'time', 'wear', 'set...","['great', 'feel', 'dont', 'time', 'wear', 'set..."
3,3,A1WMRR494NWEWV,0733001998,Amazon Shopper,"A nice moisturizer, all natural ingredients an...",4,"10 24, 2013","From the Greek island of Chios, this Mastiha b...",Mastiha Body Lotion,http://ecx.images-amazon.com/images/I/311WK5y1...,...,2,nice moisturizer natural ingredients parabens ...,"['bite', 'doesnt', 'dont', 'feel', 'ingredient...",[[-1.00264825e-01 -1.66364908e-02 4.28672917e...,[[-0.0686122 0.05983538 0.00930103 ... -0.0...,"['watereddown', 'parabens', 'ingredients', 'mo...","['use', 'natural', 'substantal', 'bite', 'pric...","['ive', 'nice', 'feel', 'dont', 'need', 'use',...","['doesnt', 'ive', 'nice', 'feel', 'dont', 'nee...","['doesnt', 'ive', 'nice', 'feel', 'dont', 'nee..."
4,4,A3V3RE4132GKRO,130414089X,Dianne K. Smith,This is by far the easiest eyeliner pen I have...,5,"06 4, 2014",Length : 13.5 cm\nColor: Black\n100% Brand new...,New Benefit Waterproof Automatic Eyeliner Pen ...,http://ecx.images-amazon.com/images/I/31zt-hSe...,...,6,far easiest eyeliner pen ever use try hundreds...,"['buy', 'easiest', 'eyeliner', 'far', 'goof', ...",[[-2.74014398e-02 -1.99812576e-02 2.64241025e...,[[-0.06259998 0.02419781 -0.03591996 ... -0.0...,"['use', 'smear', 'easiest', 'pen', 'eyeliner']","['far', 'buy', 'goof', 'price', 'try', 'use', ...","['thats', 'great', 'hundreds', 'love', 'far', ...","['thats', 'great', 'hundreds', 'love', 'far', ...","['thats', 'great', 'hundreds', 'love', 'far', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161102,161102,A2QMU8I7EED3K1,B000CIBVP8,Barbara Ortiz,Phillips Light touch is the best hair brush yo...,5,"05 7, 2013",186 Tuft

# CAMF

In [ ]:
import numpy as np
import pandas as pd
import itertools
from math import ceil, sqrt

In [ ]:
class ContextAwareMatrixFactorization:
    def __init__(self, data, user_column_name, item_column_name,
                 rating_column_name, context_column_names, features,
                 average_item_ratings=None):
        """
        Constructor for the class

        Parameters
        ----------
            data (DataFrame): the data for the algorithm to use
            user_column_name (string): Name of the user column
            item_column_name (string): Name of the item column
            rating_column_name (string): Name of the rating column
            context_column_name (list(string)): Names of the context columns.
            features (int): The number of features used in the factorization.
            average_item_ratings (dict(itemId,rating)): average rating of items
        ---------
        """
        self.data = data
        self.user_column_name = user_column_name
        self.item_column_name = item_column_name
        self.rating_column_name = rating_column_name
        self.context_column_names = context_column_names
        self.features = features
        self.unique_users = self.data[self.user_column_name].unique()
        self.unique_items = self.data[self.item_column_name].unique()
        self.unique_contexts = dict()
        for context in self.context_column_names:
            self.unique_contexts[context] = list(
                itertools.product(self.data[context].unique(),
                                  self.unique_items))
        self.users_dict = {k: v for v, k in enumerate(self.unique_users)}
        self.items_dict = {k: v for v, k in enumerate(self.unique_items)}
        self.context_dicts = dict()
        for key, value in self.unique_contexts.items():
            self.context_dicts[key] = {k: v for v, k in enumerate(value)}
        self.number_of_users = len(self.unique_users)
        self.number_of_items = len(self.unique_items)
        self.number_of_contexts = dict()
        for key, value in self.unique_contexts.items():
            self.number_of_contexts[key] = len(value)

        # Set random seed
        np.random.seed(42)
        self.user_feature_matrix = np.random.uniform(
            0, 1, (self.number_of_users, self.features))
        self.feature_item_matrix = np.random.uniform(
            0, 1, (self.features, self.number_of_items))
        self.user_biases = np.random.uniform(size=self.number_of_users)
        self.context_biases = dict()
        for key, value in self.number_of_contexts.items():
            self.context_biases[key] = np.random.uniform(size=value)

        self.average_item_ratings = average_item_ratings
        if self.average_item_ratings is None:
            self.get_average_ratings()

    def get_average_ratings(self):
        """
        Gets the average rating for each item and stores in a dictionary.
        """
        self.average_item_ratings = {}
        for _, row in self.data.iterrows():
            self.average_item_ratings[row[self.item_column_name]] = \
                self.data.loc[self.data[self.item_column_name] == row[
                    self.item_column_name]][self.rating_column_name].mean()
        return self.average_item_ratings

    def predict_ratings_for_entries(self, df):
        """
        Params:
            df (DataFrame): the dataframe containing rows of actual ratings
                for a specific user that we want predict ratings for
        Returns a dict of
            key: movieId and value: (actual_rating, predicted_rating)
        """
        predictions = {}
        for _, row in df.iterrows():
            user_index = self.users_dict[row[self.user_column_name]]
            item_index = self.items_dict[row[self.item_column_name]]
            context_indexes = dict()
            for key, value in self.context_dicts.items():
                context_indexes[key] = value[(row[key],
                                              row[self.item_column_name])]
            rating = row[self.rating_column_name]

            predicted_rating = self.predict_rating(user_index, item_index,
                                                   row[self.item_column_name],
                                                   context_indexes)
            predictions[row[self.item_column_name]] = (rating,
                                                       predicted_rating)
        return predictions

    def train(self, training_data, lrate, rterm, epochs=1000,
              batch_size=0.001):
        """
        Function that trains the model by learning the parameters
        through stochastic gradient descent
        """
        for epoch in range(epochs):
            batch = training_data.sample(
                ceil(len(training_data.index)*batch_size))

            for _, row in batch.iterrows():
                user_index = self.users_dict[row[self.user_column_name]]
                item_index = self.items_dict[row[self.item_column_name]]
                context_indexes = dict()
                for key, value in self.context_dicts.items():
                    context_indexes[key] = value[(row[key],
                                                  row[self.item_column_name])]
                rating = row[self.rating_column_name]

                predicted_rating = self.predict_rating(
                    user_index, item_index,
                    row[self.item_column_name],
                    context_indexes)
                error = rating - predicted_rating

                self.user_biases[user_index] = self.user_biases[
                    user_index] - lrate * (2 * error * (-1) +
                                           2 * rterm *
                                           self.user_biases[user_index])
                # Update the context variables
                for key, value in self.context_biases.items():
                    value[context_indexes[key]] = value[context_indexes[key]] \
                        - lrate * (2 * error * (-1) + 2 *
                                   rterm * value[context_indexes[key]])

                for k in range(self.features):
                    self.user_feature_matrix[user_index][k] = \
                        self.user_feature_matrix[user_index][k] - lrate * (
                        2 * error * (-1)
                        * self.feature_item_matrix[k][item_index] + 2
                        * rterm * self.user_feature_matrix[user_index][k])
                    self.feature_item_matrix[k][item_index] = \
                        self.feature_item_matrix[k][item_index] - lrate * (
                        2 * error * (-1)
                        * self.user_feature_matrix[user_index][k] + 2
                        * rterm * self.feature_item_matrix[k][item_index])

            if epoch % 100 == 0:
                mse, mae, rmse = self.calculate_metrics(training_data)
                print('Epoch', epoch, "-- MSE:", mse,
                      "-- RMSE:", rmse, "-- MAE:", mae)

    def calculate_metrics(self, data):
        """
        Calculates the metrics MSE, MAE and RMSE
        Parameter:
            data (DataFrame): the data to calculate the error for
        Returns: mse, mae, rmse
        """
        mse = 0
        mae = 0
        for _, row in data.iterrows():
            user_index = self.users_dict[row[self.user_column_name]]
            item_index = self.items_dict[row[self.item_column_name]]
            context_indexes = dict()
            for key, value in self.context_dicts.items():
                context_indexes[key] = value[(row[key],
                                              row[self.item_column_name])]
            rating = row[self.rating_column_name]

            predicted_rating = self.predict_rating(user_index, item_index,
                                                   row[self.item_column_name],
                                                   context_indexes)
            mse = mse + (rating - predicted_rating)**2
            mae = mae + abs(rating - predicted_rating)

        mse = mse / len(data.index)
        mae = mae / len(data.index)
        rmse = sqrt(mse)

        return mse, mae, rmse

    def calculate_precision_recall_mapk(self, test_data, k_val=10):
        """
        Parameters:
        ---------------------
        test_data (dataFrame): the test data
        k_val (int): the value of k used for the map at k calculation
        ---------------------
        Returns:
        precision, recall, mapk, ndcg
        """
        # Relevant items and user_top_k_items are used
        # for mapk calculation
        relevant_items = {}
        users_top_k_items = {}
        # predictions used for precision and recall
        predictions = []

        for _, row in test_data.iterrows():
            userId = row[self.user_column_name]
            itemId = row[self.item_column_name]
            unique_test_items = test_data[self.item_column_name].unique()

            # Get top k items
            if users_top_k_items.get(userId) is None:
                users_top_k_items[userId] = self.get_top_k_pred(
                    userId, unique_test_items, k_val)

            context_indexes = {}
            for key, value in self.context_dicts.items():
                context_indexes[key] = value[(row[key], itemId)]
            prediction = self.predict_rating(self.users_dict[userId],
                                             self.items_dict[itemId], itemId,
                                             context_indexes)
            predictions.append((userId, itemId, row[self.rating_column_name],
                                prediction, None))

            if row[self.rating_column_name] >= 3:
                if relevant_items.get(userId) is None:
                    relevant_items[userId] = [itemId]
                else:
                    relevant_items[userId].append(itemId)

        mapk = calculate_map(users_top_k_items, relevant_items, k_val)
        precision, recall = precision_recall_at_k(predictions)
        ndcg = calculate_ndcg(users_top_k_items, relevant_items)

        return precision, recall, mapk, ndcg

    def get_top_k_pred(self, userId, unique_test_items, k_val):
        predictions = {}
        lengths = list([(column_name, len(self.data[column_name].unique()))
                        for column_name in self.context_column_names])
        context_values_dict = {}
        for name in self.context_column_names:
            context_values_dict[name] = self.data[name].unique()
        counters = [0] * len(self.context_column_names)

        def nested_for_context(item_id, counters, lengths, level=0):
            if level == len(counters):
                user_index = self.users_dict[userId]
                item_index = self.items_dict[item_id]
                context_values = dict()
                for index, value in enumerate(lengths):
                    context_values[value[0]] = context_values_dict[
                        value[0]][counters[index]]
                context_indexes = dict([(key,
                                         self.context_dicts[key][(value,
                                                                  item_id)])
                                        for key, value
                                        in context_values.items()])
                predicted_rating = self.predict_rating(user_index, item_index,
                                                       item_id,
                                                       context_indexes)

                predictions[(item_id,) +
                            tuple(context_values.items())] = predicted_rating
            else:
                for _ in range(lengths[level][1]):
                    nested_for_context(item_id, counters, lengths, level + 1)
                    counters[level] += 1
                counters[level] = 0

        for item in unique_test_items:
            nested_for_context(item, counters, lengths, 0)

        sorted_predictions = sorted(predictions,
                                    key=predictions.get, reverse=True)
        top_items = [x[0] for x in sorted_predictions]
        top_items = list(dict.fromkeys(top_items))
        return top_items[:k_val]

    def predict_rating(self, user_index, item_index, item_id, context_indexes):
        predicted_rating = (np.dot(
            self.user_feature_matrix[user_index, :],
            self.feature_item_matrix[:, item_index])
            + self.average_item_ratings[item_id]
            + self.user_biases[user_index]
            + sum([value[context_indexes[key]]
                   for key, value in self.context_biases.items()]))
        return predicted_rating

In [ ]:
def calculate_map(topk_dict, relevant_dict, k_val):
    """

    Parameters
    ----------
    topk_dict: A dictionary of the top k results for each user in evaluation.
                Key should be ID. Value should be a list, comparable to the
                list used for relevant_dict.
    relevant_dict: A dictionary of all relevant items in the set for each user
                    in evaluation. The key should be the user ID, equal to
                    the keys in topk_dict.
    k_val: The number of items in each list of recommendations

    Returns the average of the sum of the averaged precisions for each user
    -------

    """
    average_precisions = []
    number_of_users = len(topk_dict)

    for userId, items in topk_dict.items():
        score = 0.0
        hit_rate = 0.0

        counter = 1
        for itemId in items:
            if userId in relevant_dict:
                if itemId in relevant_dict[userId]:
                    hit_rate += 1
                    score += hit_rate / counter
            counter += 1

        if relevant_dict.get(userId) is None:
            average_precisions.append(0.0)
        elif len(relevant_dict[userId]) < k_val:
            average_precisions.append(score / len(relevant_dict[userId]))
        else:
            average_precisions.append(score / k_val)

    return (1 / number_of_users) * sum(average_precisions)

In [ ]:
from collections import defaultdict


def precision_recall_at_k(predictions, k=10, threshold=3):
    """Precision and recall at k for each user"""

    user_estim_true = defaultdict(list)
    for user_id, _, true_rating, estimated, _ in predictions:
        user_estim_true[user_id].append((estimated, true_rating))

    precisions = dict()
    recalls = dict()

    for user_id, user_ratings in user_estim_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        number_rel = sum((true_rating >= threshold) for (_, true_rating)
                         in user_ratings)
        number_rel_rec_k = sum(((true_rating >= threshold) and
                                (estim_rating >= threshold)) for
                               (estim_rating, true_rating) in user_ratings[:k])
        precisions[user_id] = number_rel_rec_k / k
        recalls[user_id] = number_rel_rec_k / number_rel \
            if number_rel != 0 else 0

    average_precision = sum(precisions.values()) / len(precisions)

    average_recall = sum(recalls.values()) / len(recalls)

    return average_precision, average_recall

In [ ]:
import math


def calculate_ndcg(top_k_dict, relevant_dict):
    """

    Parameters
    ----------
    top_k_dict: A dictionary of the top k results for each user in evaluation.
                Key should be ID. Value should be a list, comparable to the
                list used for relevant_dict.
    relevant_dict: A dictionary of all relevant items in the set for each user
                    in evaluation. The key should be the user ID, equal to
                    the keys in topk_dict.

    Returns the normalized discounted cumulative gain for all users, where
    the gain is a binary value of 1 if the item being recommended is relevant,
    and 0 if not.
    -------

    """
    dcg, idcg, gain = 0.0, 0.0, 1
    number_of_users = len(top_k_dict)

    for key, value in top_k_dict.items():
        position = 0
        number_of_relevant_items = 0
        for val in value:
            position += 1
            if key in relevant_dict.keys():
                if val in relevant_dict[key]:
                    number_of_relevant_items += 1
                    dcg += gain / math.log2(position + 1)

        if number_of_relevant_items > 0:
            for i in range(number_of_relevant_items):
                idcg += 1 / math.log2((i+1) + 1)

    final_dcg = (1 / number_of_users) * dcg
    final_idcg = (1 / number_of_users) * idcg

    return final_dcg / final_idcg


In [ ]:
df_merge.isnull().sum()

Unnamed: 0              0
reviewerID              0
productID               0
reviewerName            0
review_lama             0
rating                  0
reviewTime              0
description             0
title                   0
imageUrl                0
salesRank               0
price                   0
related                 0
brand                   0
pos_feedback            0
neg_feedback            0
revID                   0
productID-baru          0
review                  0
candidates              0
doc_embedding           0
candidate_embeddings    0
top5                    0
top10                   0
top15                   0
top20                   0
top25                   0
dtype: int64

In [ ]:
df_first = df_merge[:50000]

Train model with top keyword

In [ ]:
train_top5_with_price = ContextAwareMatrixFactorization(df_first, 'revID', 'productID-baru','rating',['top5', 'price'], 10)

In [ ]:
train_top5_with_price.train(training_data=df_first, lrate=0.01, rterm=0.2,batch_size=0.01)

Epoch 0 -- MSE: 14.991838312529309 -- RMSE: 3.8719295335180504 -- MAE: 3.5274772197506357
Epoch 100 -- MSE: 2.901400665327576 -- RMSE: 1.7033498364480433 -- MAE: 1.2585383175158287
Epoch 200 -- MSE: 2.03395950762744 -- RMSE: 1.4261695227522708 -- MAE: 1.0644184238898873
Epoch 300 -- MSE: 1.6088945483315356 -- RMSE: 1.268422070263497 -- MAE: 0.9577318492186587
Epoch 400 -- MSE: 1.347371690087828 -- RMSE: 1.160763408317056 -- MAE: 0.8753229100904687
Epoch 500 -- MSE: 1.1594696263417121 -- RMSE: 1.0767867134868039 -- MAE: 0.8077138295042822
Epoch 600 -- MSE: 1.0110109350280978 -- RMSE: 1.0054903952938077 -- MAE: 0.7553848098954874
Epoch 700 -- MSE: 0.8936099341866457 -- RMSE: 0.9453094383251686 -- MAE: 0.7041627119228048
Epoch 800 -- MSE: 0.7930959418835776 -- RMSE: 0.8905593421460344 -- MAE: 0.6666360682297351
Epoch 900 -- MSE: 0.7009016416828995 -- RMSE: 0.8371986871005589 -- MAE: 0.6268939835852049


In [ ]:
train_top10_with_price = ContextAwareMatrixFactorization(df_first, 'revID', 'productID-baru','rating',['top10', 'price'], 10)

In [ ]:
train_top10_with_price.train(training_data=df_first, lrate=0.01, rterm=0.2,batch_size=0.01)

Epoch 0 -- MSE: 14.734441523536143 -- RMSE: 3.83854679840381 -- MAE: 3.4812614939977835
Epoch 100 -- MSE: 2.8922904070476427 -- RMSE: 1.700673515713008 -- MAE: 1.2537664658135075
Epoch 200 -- MSE: 2.021107285784601 -- RMSE: 1.4216565287665657 -- MAE: 1.0574388081581503
Epoch 300 -- MSE: 1.6115709055753995 -- RMSE: 1.2694766266361108 -- MAE: 0.9505608718423159
Epoch 400 -- MSE: 1.3534888416485173 -- RMSE: 1.1633953935135368 -- MAE: 0.8735064482308896
Epoch 500 -- MSE: 1.1648137188258278 -- RMSE: 1.0792653607087683 -- MAE: 0.8080269908018928
Epoch 600 -- MSE: 1.0085001775514837 -- RMSE: 1.0042410953309389 -- MAE: 0.7520178591062622
Epoch 700 -- MSE: 0.889506689477571 -- RMSE: 0.943136622911851 -- MAE: 0.708142503693637
Epoch 800 -- MSE: 0.7861070308581933 -- RMSE: 0.8866267708896418 -- MAE: 0.6646948238130626
Epoch 900 -- MSE: 0.7050387855182463 -- RMSE: 0.839665877309687 -- MAE: 0.6242974881346246


In [ ]:
train_top15_with_price = ContextAwareMatrixFactorization(df_first, 'revID', 'productID-baru','rating',['top15', 'price'], 10)

In [ ]:
train_top15_with_price.train(training_data=df_first, lrate=0.01, rterm=0.2,batch_size=0.01)

Epoch 0 -- MSE: 14.734441523536143 -- RMSE: 3.83854679840381 -- MAE: 3.4812614939977835
Epoch 100 -- MSE: 2.8922904070476427 -- RMSE: 1.700673515713008 -- MAE: 1.2537664658135075
Epoch 200 -- MSE: 2.021107285784601 -- RMSE: 1.4216565287665657 -- MAE: 1.0574388081581503
Epoch 300 -- MSE: 1.6115709055753995 -- RMSE: 1.2694766266361108 -- MAE: 0.9505608718423159
Epoch 400 -- MSE: 1.3534888416485173 -- RMSE: 1.1633953935135368 -- MAE: 0.8735064482308896
Epoch 500 -- MSE: 1.1648137188258278 -- RMSE: 1.0792653607087683 -- MAE: 0.8080269908018928
Epoch 600 -- MSE: 1.0085001775514837 -- RMSE: 1.0042410953309389 -- MAE: 0.7520178591062622
Epoch 700 -- MSE: 0.889506689477571 -- RMSE: 0.943136622911851 -- MAE: 0.708142503693637
Epoch 800 -- MSE: 0.7861070308581933 -- RMSE: 0.8866267708896418 -- MAE: 0.6646948238130626
Epoch 900 -- MSE: 0.7050387855182463 -- RMSE: 0.839665877309687 -- MAE: 0.6242974881346246


In [ ]:
train_top20_with_price = ContextAwareMatrixFactorization(df_first, 'revID', 'productID-baru','rating',['top20', 'price'], 10)

In [ ]:
train_top20_with_price.train(training_data=df_first, lrate=0.01, rterm=0.2,batch_size=0.01)

Epoch 0 -- MSE: 14.734441523536143 -- RMSE: 3.83854679840381 -- MAE: 3.4812614939977835
Epoch 100 -- MSE: 2.8922904070476427 -- RMSE: 1.700673515713008 -- MAE: 1.2537664658135075
Epoch 200 -- MSE: 2.021107285784601 -- RMSE: 1.4216565287665657 -- MAE: 1.0574388081581503
Epoch 300 -- MSE: 1.6115709055753995 -- RMSE: 1.2694766266361108 -- MAE: 0.9505608718423159
Epoch 400 -- MSE: 1.3534888416485173 -- RMSE: 1.1633953935135368 -- MAE: 0.8735064482308896
Epoch 500 -- MSE: 1.1648137188258278 -- RMSE: 1.0792653607087683 -- MAE: 0.8080269908018928
Epoch 600 -- MSE: 1.0085001775514837 -- RMSE: 1.0042410953309389 -- MAE: 0.7520178591062622
Epoch 700 -- MSE: 0.889506689477571 -- RMSE: 0.943136622911851 -- MAE: 0.708142503693637
Epoch 800 -- MSE: 0.7861070308581933 -- RMSE: 0.8866267708896418 -- MAE: 0.6646948238130626
Epoch 900 -- MSE: 0.7050387855182463 -- RMSE: 0.839665877309687 -- MAE: 0.6242974881346246


In [ ]:
train_top25_with_price = ContextAwareMatrixFactorization(df_first, 'revID', 'productID-baru','rating',['top25', 'price'], 10)

In [ ]:
train_top25_with_price.train(training_data=df_first, lrate=0.01, rterm=0.2,batch_size=0.01)

Epoch 0 -- MSE: 14.734441523536143 -- RMSE: 3.83854679840381 -- MAE: 3.4812614939977835
Epoch 100 -- MSE: 2.8922904070476427 -- RMSE: 1.700673515713008 -- MAE: 1.2537664658135075
Epoch 200 -- MSE: 2.021107285784601 -- RMSE: 1.4216565287665657 -- MAE: 1.0574388081581503
Epoch 300 -- MSE: 1.6115709055753995 -- RMSE: 1.2694766266361108 -- MAE: 0.9505608718423159
Epoch 400 -- MSE: 1.3534888416485173 -- RMSE: 1.1633953935135368 -- MAE: 0.8735064482308896
Epoch 500 -- MSE: 1.1648137188258278 -- RMSE: 1.0792653607087683 -- MAE: 0.8080269908018928
Epoch 600 -- MSE: 1.0085001775514837 -- RMSE: 1.0042410953309389 -- MAE: 0.7520178591062622
Epoch 700 -- MSE: 0.889506689477571 -- RMSE: 0.943136622911851 -- MAE: 0.708142503693637
Epoch 800 -- MSE: 0.7861070308581933 -- RMSE: 0.8866267708896418 -- MAE: 0.6646948238130626
Epoch 900 -- MSE: 0.7050387855182463 -- RMSE: 0.839665877309687 -- MAE: 0.6242974881346246


Train with no keyword and no other context

In [ ]:
train_no_keyword = ContextAwareMatrixFactorization(df_first, 'revID', 'productID-baru','rating',['price'], 10)

In [ ]:
train_no_keyword.train(training_data=df_first, lrate=0.01, rterm=0.2,batch_size=0.01)

Epoch 0 -- MSE: 11.609263752594421 -- RMSE: 3.4072369674847125 -- MAE: 3.019031238863747
Epoch 100 -- MSE: 2.5738666721202197 -- RMSE: 1.6043274828164664 -- MAE: 1.193153188803099
Epoch 200 -- MSE: 1.9308003324252279 -- RMSE: 1.3895324150322035 -- MAE: 1.039162371968391
Epoch 300 -- MSE: 1.6344936863156496 -- RMSE: 1.2784731856068197 -- MAE: 0.9597499635029806
Epoch 400 -- MSE: 1.4419566002630553 -- RMSE: 1.2008149733672775 -- MAE: 0.9116317895303614
Epoch 500 -- MSE: 1.3002091683957744 -- RMSE: 1.1402671478192181 -- MAE: 0.8678808171041209
Epoch 600 -- MSE: 1.1817769117914767 -- RMSE: 1.0870956313919566 -- MAE: 0.8186113339454764
Epoch 700 -- MSE: 1.0877176685591747 -- RMSE: 1.0429370395949962 -- MAE: 0.7906407247516184
Epoch 800 -- MSE: 1.0094133346064087 -- RMSE: 1.00469564277268 -- MAE: 0.7483272091323824
Epoch 900 -- MSE: 0.9231638988485814 -- RMSE: 0.9608141853910055 -- MAE: 0.7305931746128894


train with all keyword used

In [ ]:
train_with_candidates = ContextAwareMatrixFactorization(df_first, 'revID', 'productID-baru','rating',['price', 'candidates'], 10)

In [ ]:
train_with_candidates.train(training_data=df_first, lrate=0.01, rterm=0.2,batch_size=0.01)

Epoch 0 -- MSE: 14.51613007894458 -- RMSE: 3.81000394736601 -- MAE: 3.4275879918909653
Epoch 100 -- MSE: 2.911698694619376 -- RMSE: 1.7063700344940942 -- MAE: 1.26074120309301
Epoch 200 -- MSE: 2.0293926788600274 -- RMSE: 1.4245675409962237 -- MAE: 1.0616706110641192
Epoch 300 -- MSE: 1.6080123457367343 -- RMSE: 1.268074266648738 -- MAE: 0.9499134905897401
Epoch 400 -- MSE: 1.3426727712252235 -- RMSE: 1.1587375765138643 -- MAE: 0.8745393266889737
Epoch 500 -- MSE: 1.1542243480106256 -- RMSE: 1.0743483364396418 -- MAE: 0.8032571072765211
Epoch 600 -- MSE: 1.0048114484592483 -- RMSE: 1.0024028374158007 -- MAE: 0.7523823759194763
Epoch 700 -- MSE: 0.8863482882113322 -- RMSE: 0.9414607204824491 -- MAE: 0.7064735811260969
Epoch 800 -- MSE: 0.7885803713415335 -- RMSE: 0.888020479122826 -- MAE: 0.6669746899685693
Epoch 900 -- MSE: 0.699995162488374 -- RMSE: 0.8366571355629342 -- MAE: 0.6303117790344636


In [ ]:
df_cluster = pd.read_csv('export_with_cluster_k3.csv')

In [ ]:
df_cluster.rename(columns = {'bowLabel':'cluster_group_k3'}, inplace = True)

In [ ]:
df_second = df_cluster[:50000]

train with cluster

In [ ]:
train_with_cluster_only = ContextAwareMatrixFactorization(df_cluster, 'revID', 'productID-baru','rating',['price', 'cluster_group_k3'], 10)

In [ ]:
train_with_cluster_only.train(training_data=df_cluster, lrate=0.01, rterm=0.2,batch_size=0.01)

Epoch 0 -- MSE: 14.972994251024192 -- RMSE: 3.869495348365752 -- MAE: 3.5062929077241947
Epoch 100 -- MSE: 2.9168009520102105 -- RMSE: 1.7078644419303923 -- MAE: 1.2737112677474058
Epoch 200 -- MSE: 2.09654199314782 -- RMSE: 1.4479440573267395 -- MAE: 1.0838381399166739
Epoch 300 -- MSE: 1.725330315117476 -- RMSE: 1.3135182964532606 -- MAE: 0.9928102904227757
Epoch 400 -- MSE: 1.497287235114161 -- RMSE: 1.223636888588343 -- MAE: 0.9235535192647075
Epoch 500 -- MSE: 1.3376728766576507 -- RMSE: 1.1565780893038096 -- MAE: 0.870825829073069
Epoch 600 -- MSE: 1.2019576146657351 -- RMSE: 1.0963382756547977 -- MAE: 0.8303017911622168
Epoch 700 -- MSE: 1.1044145160193597 -- RMSE: 1.0509112788524821 -- MAE: 0.7929407300534226
Epoch 800 -- MSE: 1.0071274549468523 -- RMSE: 1.0035573999263083 -- MAE: 0.7534057929214973
Epoch 900 -- MSE: 0.9269991931397156 -- RMSE: 0.962807973138837 -- MAE: 0.7233751368742838


In [ ]:
train_with_cluster_and_top5 = ContextAwareMatrixFactorization(df_second, 'revID', 'productID-baru','rating',['price', 'cluster_group_k3', 'top5'], 10)

In [ ]:
train_with_cluster_and_top5.train(training_data=df_second, lrate=0.01, rterm=0.2,batch_size=0.01)

Epoch 0 -- MSE: 17.65607454413597 -- RMSE: 4.20191320045238 -- MAE: 3.8244780514511767
Epoch 100 -- MSE: 3.038571408924008 -- RMSE: 1.7431498526873723 -- MAE: 1.2810842947752161
Epoch 200 -- MSE: 2.067896153239956 -- RMSE: 1.4380181338355773 -- MAE: 1.061872894800187
Epoch 300 -- MSE: 1.6287991154465824 -- RMSE: 1.2762441441380181 -- MAE: 0.9375215769273897
Epoch 400 -- MSE: 1.3532883149829797 -- RMSE: 1.1633092086728187 -- MAE: 0.8726834423580049
Epoch 500 -- MSE: 1.1533724923303696 -- RMSE: 1.073951810990777 -- MAE: 0.808827974075276
Epoch 600 -- MSE: 1.0037126394817164 -- RMSE: 1.0018545999703332 -- MAE: 0.7531709602237151
Epoch 700 -- MSE: 0.8911467226373642 -- RMSE: 0.9440056793459265 -- MAE: 0.7117214129211588
Epoch 800 -- MSE: 0.782844716250994 -- RMSE: 0.8847851243386691 -- MAE: 0.6613411458679603
Epoch 900 -- MSE: 0.6938387974191483 -- RMSE: 0.8329698658529902 -- MAE: 0.619252616881043


In [ ]:
train_with_cluster_and_top10 = ContextAwareMatrixFactorization(df_second, 'revID', 'productID-baru','rating',['price', 'cluster_group_k3', 'top10'], 10)

In [ ]:
train_with_cluster_and_top10.train(training_data=df_second, lrate=0.01, rterm=0.2,batch_size=0.01)

Epoch 0 -- MSE: 17.691975879958488 -- RMSE: 4.2061830535484885 -- MAE: 3.8234949552145547
Epoch 100 -- MSE: 3.0494852255563125 -- RMSE: 1.7462775339436492 -- MAE: 1.277666528093544
Epoch 200 -- MSE: 2.0854160261398844 -- RMSE: 1.444096958704603 -- MAE: 1.0765349820481513
Epoch 300 -- MSE: 1.6205110687346618 -- RMSE: 1.272992957064045 -- MAE: 0.9562173703489899
Epoch 400 -- MSE: 1.3550550781803439 -- RMSE: 1.1640683305460826 -- MAE: 0.8669377206831628
Epoch 500 -- MSE: 1.1527157604002976 -- RMSE: 1.0736460126132343 -- MAE: 0.8069491244580513
Epoch 600 -- MSE: 1.0056990977246894 -- RMSE: 1.0028455004260075 -- MAE: 0.7535694169170679
Epoch 700 -- MSE: 0.8810706160938543 -- RMSE: 0.938653618804005 -- MAE: 0.702787082926346
Epoch 800 -- MSE: 0.7792144410372072 -- RMSE: 0.8827312394139041 -- MAE: 0.6606692530140879
Epoch 900 -- MSE: 0.6936443622250336 -- RMSE: 0.8328531456535622 -- MAE: 0.620489990632475


In [ ]:
train_with_cluster_and_top15 = ContextAwareMatrixFactorization(df_second, 'revID', 'productID-baru','rating',['price', 'cluster_group_k3', 'top15'], 10)

In [ ]:
train_with_cluster_and_top15.train(training_data=df_second, lrate=0.01, rterm=0.2,batch_size=0.01)

Epoch 0 -- MSE: 17.691975879958488 -- RMSE: 4.2061830535484885 -- MAE: 3.8234949552145547
Epoch 100 -- MSE: 3.0494852255563125 -- RMSE: 1.7462775339436492 -- MAE: 1.277666528093544
Epoch 200 -- MSE: 2.0854160261398844 -- RMSE: 1.444096958704603 -- MAE: 1.0765349820481513
Epoch 300 -- MSE: 1.6205110687346618 -- RMSE: 1.272992957064045 -- MAE: 0.9562173703489899
Epoch 400 -- MSE: 1.3550550781803439 -- RMSE: 1.1640683305460826 -- MAE: 0.8669377206831628
Epoch 500 -- MSE: 1.1527157604002976 -- RMSE: 1.0736460126132343 -- MAE: 0.8069491244580513
Epoch 600 -- MSE: 1.0056990977246894 -- RMSE: 1.0028455004260075 -- MAE: 0.7535694169170679
Epoch 700 -- MSE: 0.8810706160938543 -- RMSE: 0.938653618804005 -- MAE: 0.702787082926346
Epoch 800 -- MSE: 0.7792144410372072 -- RMSE: 0.8827312394139041 -- MAE: 0.6606692530140879
Epoch 900 -- MSE: 0.6936443622250336 -- RMSE: 0.8328531456535622 -- MAE: 0.620489990632475


In [ ]:
train_with_cluster_and_top20 = ContextAwareMatrixFactorization(df_second, 'revID', 'productID-baru','rating',['price', 'cluster_group_k3', 'top20'], 10)

In [ ]:
train_with_cluster_and_top20.train(training_data=df_second, lrate=0.01, rterm=0.2,batch_size=0.01)

Epoch 0 -- MSE: 17.691975879958488 -- RMSE: 4.2061830535484885 -- MAE: 3.8234949552145547
Epoch 100 -- MSE: 3.0494852255563125 -- RMSE: 1.7462775339436492 -- MAE: 1.277666528093544
Epoch 200 -- MSE: 2.0854160261398844 -- RMSE: 1.444096958704603 -- MAE: 1.0765349820481513
Epoch 300 -- MSE: 1.6205110687346618 -- RMSE: 1.272992957064045 -- MAE: 0.9562173703489899
Epoch 400 -- MSE: 1.3550550781803439 -- RMSE: 1.1640683305460826 -- MAE: 0.8669377206831628
Epoch 500 -- MSE: 1.1527157604002976 -- RMSE: 1.0736460126132343 -- MAE: 0.8069491244580513
Epoch 600 -- MSE: 1.0056990977246894 -- RMSE: 1.0028455004260075 -- MAE: 0.7535694169170679
Epoch 700 -- MSE: 0.8810706160938543 -- RMSE: 0.938653618804005 -- MAE: 0.702787082926346
Epoch 800 -- MSE: 0.7792144410372072 -- RMSE: 0.8827312394139041 -- MAE: 0.6606692530140879
Epoch 900 -- MSE: 0.6936443622250336 -- RMSE: 0.8328531456535622 -- MAE: 0.620489990632475


In [ ]:
train_with_cluster_and_top25 = ContextAwareMatrixFactorization(df_second, 'revID', 'productID-baru','rating',['price', 'cluster_group_k3', 'top25'], 10)

In [ ]:
train_with_cluster_and_top25.train(training_data=df_second, lrate=0.01, rterm=0.2,batch_size=0.01)

Epoch 0 -- MSE: 17.691975879958488 -- RMSE: 4.2061830535484885 -- MAE: 3.8234949552145547
Epoch 100 -- MSE: 3.0494852255563125 -- RMSE: 1.7462775339436492 -- MAE: 1.277666528093544
Epoch 200 -- MSE: 2.0854160261398844 -- RMSE: 1.444096958704603 -- MAE: 1.0765349820481513
Epoch 300 -- MSE: 1.6205110687346618 -- RMSE: 1.272992957064045 -- MAE: 0.9562173703489899
Epoch 400 -- MSE: 1.3550550781803439 -- RMSE: 1.1640683305460826 -- MAE: 0.8669377206831628
Epoch 500 -- MSE: 1.1527157604002976 -- RMSE: 1.0736460126132343 -- MAE: 0.8069491244580513
Epoch 600 -- MSE: 1.0056990977246894 -- RMSE: 1.0028455004260075 -- MAE: 0.7535694169170679
Epoch 700 -- MSE: 0.8810706160938543 -- RMSE: 0.938653618804005 -- MAE: 0.702787082926346
Epoch 800 -- MSE: 0.7792144410372072 -- RMSE: 0.8827312394139041 -- MAE: 0.6606692530140879
Epoch 900 -- MSE: 0.6936443622250336 -- RMSE: 0.8328531456535622 -- MAE: 0.620489990632475


In [ ]:
train_with_cluster_and_candidates = ContextAwareMatrixFactorization(df_second, 'revID', 'productID-baru','rating',['price', 'cluster_group_k3', 'candidates'], 10)

In [ ]:
train_with_cluster_and_candidates.train(training_data=df_second, lrate=0.01, rterm=0.2,batch_size=0.01)

Epoch 0 -- MSE: 17.674230458435638 -- RMSE: 4.20407307957838 -- MAE: 3.809684787332368
Epoch 100 -- MSE: 3.026808642051914 -- RMSE: 1.7397725834292004 -- MAE: 1.2784689845151505
Epoch 200 -- MSE: 2.0834843403560024 -- RMSE: 1.4434279823933034 -- MAE: 1.067761868017443
Epoch 300 -- MSE: 1.6421835131022882 -- RMSE: 1.2814770825505575 -- MAE: 0.9665011121657323
Epoch 400 -- MSE: 1.352301481271709 -- RMSE: 1.1628849819615477 -- MAE: 0.8756873390972677
Epoch 500 -- MSE: 1.1634812848936027 -- RMSE: 1.0786478966250306 -- MAE: 0.8143525537781184
Epoch 600 -- MSE: 0.9948947144667544 -- RMSE: 0.9974440908977076 -- MAE: 0.7515494962225069
Epoch 700 -- MSE: 0.884306983491312 -- RMSE: 0.9403759798566274 -- MAE: 0.703525383485955
Epoch 800 -- MSE: 0.7815133194814264 -- RMSE: 0.8840324199266826 -- MAE: 0.6662222759740393
Epoch 900 -- MSE: 0.7067482186324825 -- RMSE: 0.8406831856487214 -- MAE: 0.6297467846102606
